# preprocessingパッケージについて

**概要**<br>
日本語の自然言語処理における前処理の関数をまとめたモジュールです<br><br>
**一般的な前処理の流れ**<br>
（np.nanの文章、もしくは変な読み込まれ方をした文を修正）<br>
ストップワードの除去<br>
数字の正規化<br>
（表現の正規化）<br>
分かち書き<br>
コーパス作成<br>
頻度を元に語句を除去<br>

**構成**<br>
preprocess.py : 以下のモジュールを使った前処理クラスを提供<br>
remove_stopwords.py : ストップワード（分析する上で使わない言葉）の削除<br>
stopwords_config.py : ストップワードのパターンを保存しておく<br>
normalize_number.py : 数字を正規化<br>
mecab_handlers.py : mecabモジュールの読み込み・分かち書き関数<br>
make_corpus.py : gensimを使って辞書やコーパスを作成

In [1]:
doc = "朝９時に目覚めると、1111円札が99枚見つかって、１６時まで寝てしまいました。"

## remove_stopwords.py, stopwords_config.py

In [2]:
# ストップワードのパターン
from stopwords_config import stop_words_1

In [3]:
print(stop_words_1)

['。', '、', '・', ',', '*', '＊', '（', '）', '(', ')', '!', '！', '?', '？', '[', ']', '「', '」', '『', '』', '【', '】', '■', '□', '●', '○', '◆', '◇', '★', '☆', '~', '〜', '"', '”', '\n', '/']


In [4]:
# ストップワードを削除
from remove_stopwords import remove_stopwords

In [5]:
removed = remove_stopwords(doc=doc, stopwords=stop_words_1)
print(removed)

朝９時に目覚めると1111円札が99枚見つかって１６時まで寝てしまいました


## normalize_number.py

In [6]:
# 数字を全て同じ値にする
from normalize_number import normalize_number

In [7]:
normalized = normalize_number(doc=doc, replace="0")
print(normalized)

朝0時に目覚めると、0円札が0枚見つかって、0時まで寝てしまいました。


In [8]:
# 特定のパターンで出現した数字を同じ値に変換
from normalize_number import normalize_number_specified_pattern

In [9]:
normalized = normalize_number_specified_pattern(doc=doc, patterns={"prefix":["朝"], "suffix":[]}, replace="hoge")
print(normalized)

朝hoge時に目覚めると、1111円札が99枚見つかって、１６時まで寝てしまいました。


In [10]:
normalized = normalize_number_specified_pattern(doc=doc, patterns={"prefix":["朝"], "suffix":["時", "枚"]}, replace="hoge")
print(normalized)

朝hoge時に目覚めると、1111円札がhoge枚見つかって、hoge時まで寝てしまいました。


## mecab_handlers.py

In [11]:
# 分かち書き
from mecab_handlers import doc_to_words

In [12]:
split = doc_to_words(doc=doc)
print(split)

['朝', '９', '時', 'に', '目覚める', 'と', '、', '1111', '円', '札', 'が', '99', '枚', '見つかっ', 'て', '、', '１', '６', '時', 'まで', '寝', 'て', 'しまい', 'まし', 'た', '。']


In [13]:
split_only_noun = doc_to_words(doc, parts=["名詞"])
split_noun_and_verb = doc_to_words(doc, parts=["名詞", "動詞"])
print(split_only_noun)
print(split_noun_and_verb)

['朝', '９', '時', '1111', '円', '札', '99', '枚', '１', '６', '時']
['朝', '９', '時', '目覚める', '1111', '円', '札', '99', '枚', '見つかっ', '１', '６', '時', '寝', 'しまい']


## make_corpus.py

In [14]:
# gensimで辞書を作成
import sys
sys.path.append("../")
from preprocessing.make_corpus import make_dictionary

In [15]:
docs = ["今日はいい天気", "東京の渋谷のカフェ", "昆虫博士と茂木健一郎"]
words_list = [doc_to_words(doc, parts=["名詞"]) for doc in docs]
print(words_list)

[['今日', '天気'], ['東京', '渋谷', 'カフェ'], ['昆虫', '博士', '茂木', '健一郎']]


In [16]:
dictionary = make_dictionary(words_list)
print(dictionary)

Dictionary(9 unique tokens: ['今日', '天気', 'カフェ', '東京', '渋谷']...)


In [17]:
for i in dictionary.items():
    print(i)

(0, '今日')
(1, '天気')
(2, 'カフェ')
(3, '東京')
(4, '渋谷')
(5, '健一郎')
(6, '博士')
(7, '昆虫')
(8, '茂木')


In [18]:
dictionary.id2token[5]

'健一郎'

In [19]:
dictionary.token2id["カフェ"]

2

## make_corpus.py

In [20]:
# 事前にgensimで作った辞書を元に、コーパスを作成
sys.path.append("../")
from preprocessing.make_corpus import  make_corpus

In [21]:
corpus = make_corpus(dictionary=dictionary, words_list=words_list)
print(corpus)

[[(0, 1), (1, 1)], [(2, 1), (3, 1), (4, 1)], [(5, 1), (6, 1), (7, 1), (8, 1)]]


In [22]:
for i, l in enumerate(corpus):
    for tup in l:
        word_id = tup[0]
        word = dictionary.id2token[word_id]
        n = tup[1]
        print("{}番目の文章で、単語「{}」が{}回出現しました".format(i, word, n))

0番目の文章で、単語「今日」が1回出現しました
0番目の文章で、単語「天気」が1回出現しました
1番目の文章で、単語「カフェ」が1回出現しました
1番目の文章で、単語「東京」が1回出現しました
1番目の文章で、単語「渋谷」が1回出現しました
2番目の文章で、単語「健一郎」が1回出現しました
2番目の文章で、単語「博士」が1回出現しました
2番目の文章で、単語「昆虫」が1回出現しました
2番目の文章で、単語「茂木」が1回出現しました


## preprocess.py

In [23]:
from preprocessing import Preprocessing

In [24]:
print(docs)

['今日はいい天気', '東京の渋谷のカフェ', '昆虫博士と茂木健一郎']


In [25]:
process = Preprocessing(docs)